# Using generators to get numpy chunks out of TAQ data

In [28]:
from statistics import mode

def print_stats(chunk):
    #find the max bid price
    max_price = max(chunk['Bid_Price'])

    #find the min bid price
    min_price = min(chunk['Bid_Price'])

    #find the mean of bid price
    avg_price = np.mean(chunk['Bid_Price'])

    #find the mod of bid price
    try:
        mod_price = mode(chunk['Bid_Price'])
    except StatisticsError:
        mod_price = np.nan
        
    #find the sd of bid price
    sd_price = np.std(chunk['Bid_Price'])

    print("Max bid price: ", max_price, "\n", "Min bid price: ", min_price, "\n", 
          "Mean bid price: ", avg_price, "\n", "Mod bid price: ", mod_price, "\n",
          "Standard deviation bid price: ", sd_price)

In [1]:
from glob import glob
from glob import glob
from itertools import (islice, zip_longest)
from collections import Counter
import numpy as np
from collections import Counter
from itertools import islice
import raw_taq_new

In [2]:
# You can run this if you update the raw_taq.py file
from importlib import reload
reload(raw_taq_new)

<module 'raw_taq_new' from 'c:\\Users\\jie\\Documents\\GitHub\\dlab-finance\\basic-taq\\raw_taq_new.py'>

### Here, we grab whatever BBO file we can find

In [16]:
fname = '../local_data/EQY_US_ALL_BBO_20140206.zip'
chunks = raw_taq_new.TAQ2Chunks(fname,chunksize=10, do_process_chunk=False)

In [17]:
chunk_gen = chunks.convert_taq()

In [18]:
type(chunk_gen)

generator

In [19]:
# You can get one chunk this way
chunk = next(chunk_gen)
chunk[0]

TypeError: 'NoneType' object is not subscriptable

In [20]:
# If you want just the type
chunk.dtype

dtype([('hour', 'S2'), ('minute', 'S2'), ('msec', 'S5'), ('Exchange', 'S1'), ('Symbol_root', 'S6'), ('Symbol_suffix', 'S10'), ('Bid_Price', 'S11'), ('Bid_Size', 'S7'), ('Ask_Price', 'S11'), ('Ask_Size', 'S7'), ('Quote_Condition', 'S1'), ('Market_Maker', 'S4'), ('Bid_Exchange', 'S1'), ('Ask_Exchange', 'S1'), ('Sequence_Number', 'S16'), ('National_BBO_Ind', 'S1'), ('NASDAQ_BBO_Ind', 'S1'), ('Quote_Cancel_Correction', 'S1'), ('Source_of_Quote', 'S1'), ('Retail_Interest_Indicator_RPI', 'S1'), ('Short_Sale_Restriction_Indicator', 'S1'), ('LULD_BBO_Indicator_CQS', 'S1'), ('LULD_BBO_Indicator_UTP', 'S1'), ('FINRA_ADF_MPID_Indicator', 'S1'), ('SIP_generated_Message_Identifier', 'S1'), ('National_BBO_LULD_Indicator', 'S1'), ('newline', 'S2')])

In [21]:
# Numpy record arrays support string indexing to get columns
print(chunk['Bid_Price'])
print(chunk["Ask_Price"])

[b'00000000000' b'00000000000' b'00000000000' b'00000419000' b'00000540700'
 b'00000574300' b'00000560700' b'00000000000' b'00000000000' b'00000000000']
[b'00000000000' b'00000729400' b'00000607600' b'00000607600' b'00000607600'
 b'00000607600' b'00000640000' b'00000632900' b'00000000000' b'00000633000']


In [1]:
from glob import glob
from glob import glob
from itertools import (islice, zip_longest)
from collections import Counter
import numpy as np
from collections import Counter
from itertools import islice
import raw_taq

In [2]:
from importlib import reload
reload(raw_taq)

<module 'raw_taq' from 'c:\\Users\\jie\\Documents\\GitHub\\dlab-finance\\basic-taq\\raw_taq.py'>

In [3]:
fname = '../local_data/EQY_US_ALL_BBO_20140206.zip'
chunks = raw_taq.TAQ2Chunks(fname,chunksize=10, do_process_chunk=True)

In [ ]:
%time chunks.to_hdf5()

In [4]:
import tables as tb
h5 = tb.open_file('20140206.h5', title='TAQ',mode='w', filters=tb.Filters(complevel=8, complib='blosc:lz4hc', fletcher32=True) )
table = h5.create_table('/', 'daily_quotes', description = chunks.bytes_spec.pytables_desc, expectedrows=chunks.numlines)

table.append(next(chunks.iter_))


In [7]:
print(chunks.bytes_spec.initial_dtype)

[('hour', 'S2'), ('minute', 'S2'), ('msec', 'S5'), ('Exchange', 'S1'), ('Symbol_root', 'S6'), ('Symbol_suffix', 'S10'), ('Bid_Price', 'S11'), ('Bid_Size', 'S7'), ('Ask_Price', 'S11'), ('Ask_Size', 'S7'), ('Quote_Condition', 'S1'), ('Market_Maker', 'S4'), ('Bid_Exchange', 'S1'), ('Ask_Exchange', 'S1'), ('Sequence_Number', 'S16'), ('National_BBO_Ind', 'S1'), ('NASDAQ_BBO_Ind', 'S1'), ('Quote_Cancel_Correction', 'S1'), ('Source_of_Quote', 'S1'), ('Retail_Interest_Indicator_RPI', 'S1'), ('Short_Sale_Restriction_Indicator', 'S1'), ('LULD_BBO_Indicator_CQS', 'S1'), ('LULD_BBO_Indicator_UTP', 'S1'), ('FINRA_ADF_MPID_Indicator', 'S1'), ('SIP_generated_Message_Identifier', 'S1'), ('National_BBO_LULD_Indicator', 'S1'), ('newline', 'S2')]


In [17]:
chunks.bytes_spec.pytables_desc

{'Ask_Exchange': StringCol(itemsize=1, shape=(), dflt=b'', pos=14),
 'Ask_Price': Float64Col(shape=(), dflt=0.0, pos=9),
 'Ask_Size': Int32Col(shape=(), dflt=0, pos=10),
 'Bid_Exchange': StringCol(itemsize=1, shape=(), dflt=b'', pos=13),
 'Bid_Price': Float64Col(shape=(), dflt=0.0, pos=7),
 'Bid_Size': Int32Col(shape=(), dflt=0, pos=8),
 'Exchange': StringCol(itemsize=1, shape=(), dflt=b'', pos=4),
 'FINRA_ADF_MPID_Indicator': StringCol(itemsize=1, shape=(), dflt=b'', pos=24),
 'LULD_BBO_Indicator_CQS': StringCol(itemsize=1, shape=(), dflt=b'', pos=22),
 'LULD_BBO_Indicator_UTP': StringCol(itemsize=1, shape=(), dflt=b'', pos=23),
 'Market_Maker': StringCol(itemsize=4, shape=(), dflt=b'', pos=12),
 'NASDAQ_BBO_Ind': StringCol(itemsize=1, shape=(), dflt=b'', pos=17),
 'National_BBO_Ind': StringCol(itemsize=1, shape=(), dflt=b'', pos=16),
 'National_BBO_LULD_Indicator': StringCol(itemsize=1, shape=(), dflt=b'', pos=26),
 'Quote_Cancel_Correction': StringCol(itemsize=1, shape=(), dflt=b'',

In [8]:
h5 = tb.open_file('./20140206.h5')
h5.root.daily_quotes[:]

array([ (1391677200.901, 4, 0, 901, b'P', b'A     ', b'          ', 0.0, 0, 0.0, 0, b'R', b'    ', b'P', b'P', 14, b'2', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688108.081, 7, 53, 81, b'P', b'A     ', b'          ', 0.0, 0, 72.94, 27, b'R', b'    ', b'P', b'P', 76255, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688108.085, 7, 53, 85, b'P', b'A     ', b'          ', 0.0, 0, 60.76, 10, b'R', b'    ', b'P', b'P', 76256, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688108.089, 7, 53, 89, b'P', b'A     ', b'          ', 41.9, 27, 60.76, 10, b'R', b'    ', b'P', b'P', 76257, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688108.094, 7, 53, 94, b'P', b'A     ', b'          ', 54.07, 27, 60.76, 10, b'R', b'    ', b'P', b'P', 76258, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688036.249, 7, 56, 4249, b'P', b'A     ', b'         

# Goal: Compute some summary statistics across a few securities in the TAQ file

Processing an entire TAQ file will take a long time. So, maybe just run through the chunks for the first two securities (you can then exit out of a loop once you see the third security / symbol).

## A complete approach

In [10]:
chunk.dtype

dtype([('Exchange', 'S1'), ('Symbol', 'S16'), ('Bid_Price', '<f8'), ('Bid_Size', '<i4'), ('Ask_Price', '<f8'), ('Ask_Size', '<i4'), ('Quote_Condition', 'S1'), ('Bid_Exchange', 'S1'), ('Ask_Exchange', 'S1'), ('Sequence_Number', '<i8'), ('National_BBO_Ind', 'S1'), ('NASDAQ_BBO_Ind', 'S1'), ('Quote_Cancel_Correction', 'S1'), ('Source_of_Quote', 'S1'), ('Retail_Interest_Indicator_RPI', 'S1'), ('Short_Sale_Restriction_Indicator', 'S1'), ('LULD_BBO_Indicator_CQS', 'S1'), ('LULD_BBO_Indicator_UTP', 'S1'), ('FINRA_ADF_MPID_Indicator', 'S1'), ('SIP_generated_Message_Identifier', 'S1'), ('National_BBO_LULD_Indicator', 'S1'), ('Time', '<f8')])

In [8]:
fname = glob('../local_data/EQY_US_ALL_BBO_*.zip')[0]
local_taq = raw_taq.TAQ2Chunks(fname)

chunk_gen = local_taq.convert_taq(20)
first_chunk = next(chunk_gen)
curr_symbol = first_chunk['Symbol_root'][0]

accum = pd.DataFrame(first_chunk)

processed_symbols = 0

for chunk in chunk_gen:
    where_symbol = curr_symbol == chunk['Symbol_root']
    if where_symbol.all():
        accum.append(pd.DataFrame(chunk))
    else:
        same = chunk[where_symbol]
        accum.append(pd.DataFrame(same))
        
        # Compute the stats
        print('Current symbol:', curr_symbol, len(curr_symbol), 'records')
        print_stats(accum)
        processed_symbols += 1
        if processed_symbols > 3:
            break
        
        diff = chunk[~where_symbol]
        accum = pd.DataFrame(diff)
        curr_symbol = accum.Symbol_root[0]

In [9]:
b'AA              ' == b'AA              '

True

# some simple examples of how generator functions work

In [16]:
def simple_fun(l):
    for item in l:
        yield item

In [17]:
simple_gen = simple_fun(['a', 'b', 1, 2])

In [18]:
type(simple_gen)

generator

In [19]:
next(simple_gen)

'a'

In [20]:
for item in simple_fun(['a', 'b', 1, 2]):
    print(item)

a
b
1
2


Running multiply BBO file

In [5]:
import raw_taq
from glob import glob

In [6]:
fname = glob('../local_data/EQY_US_ALL_BBO_201501*.zip')
print(fname)

[]
